<a href="https://colab.research.google.com/github/nagae/ICL_B_2021/blob/main/case_death_rate_weekly_growth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 感染死亡率(14日ラグ)とその週間成長率を求める

感染死亡率として，以下を定義する：
$$
\begin{equation}
t\text{日目感染死亡率} = \frac{t\text{日目累積死亡数}}{(t-14)\text{日目累積感染数}}
\end{equation}
$$
ただし, $t<14$ の場合は未定義( `NaN` )とする． データフレーム`global_df` の各国についてこれを計算したものを `case_death_rate` という列に加えるには，下記のようにする：
```
total_deaths = global_df.groupby("location")["total_deaths"].shift() # 各国の累積死亡数
total_cases14 = global_df.groupby("location")["total_cases"].shift(14) # 各国の14日前累積死亡数
global_df["case_death_rate"] = total_deaths/total_cases14 # 14日前感染数に対する死亡数
```

In [1]:
# 必要なモジュールのインポート
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt, dates as mdates
import datetime
# CSVをデータフレームに読み込む（ちょっと重い）
global_df = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')
# 年-月-日の文字列で記載されている年月日を datetime 型に変更
global_df["date"]=pd.to_datetime(global_df["date"], format="%Y-%m-%d")
# 県別でも扱ったマルチインデックスとして整形
global_df = global_df.set_index(["location", "date"])
global_df # global_df の概観を表示

iso_code  ... human_development_index
location    date                 ...                        
Afghanistan 2020-02-24      AFG  ...                   0.511
            2020-02-25      AFG  ...                   0.511
            2020-02-26      AFG  ...                   0.511
            2020-02-27      AFG  ...                   0.511
            2020-02-28      AFG  ...                   0.511
...                         ...  ...                     ...
Zimbabwe    2021-05-18      ZWE  ...                   0.571
            2021-05-19      ZWE  ...                   0.571
            2021-05-20      ZWE  ...                   0.571
            2021-05-21      ZWE  ...                   0.571
            2021-05-22      ZWE  ...                   0.571

[90532 rows x 57 columns]

In [18]:
# 感染から死亡までに14日間のラグがあるとして求めた感染死亡率をデータフレームに追加する
total_deaths = global_df.groupby("location")["total_deaths"].shift()
total_cases14 = global_df.groupby("location")["total_cases"].shift(14)
global_df["case_death_rate"] = total_deaths/total_cases14

# 国名でない地域名
area_list = ['World', 'Asia', 'Europe', 'North America', 'European Union', 'South America', 'Africa']

# 地域名
# 感染死亡率の週間成長率を求める
case_death_rate0 = global_df.groupby("location")["case_death_rate"].shift(0) # 現在の感染死亡率
case_death_rate7 = global_df.groupby("location")["case_death_rate"].shift(7) # 7日前の感染死亡率
global_df["case_death_rate_weekly_growth"] = (case_death_rate0 - case_death_rate7)/case_death_rate7

rank = 30 # 上位何位までを表示するか
CDR_WG = global_df.groupby("location")["case_death_rate_weekly_growth"].last() # 直近の感染死亡率の週間成長率
CDR_WG = CDR_WG.drop(index=area_list) # インデックスが国名ではなく地域となっている行を取り除く
CDR_WG = CDR_WG.sort_values(ascending=False).head(rank) # 残った要素を降順に並べ替え，rank 個の上位要素を取り出す

CDR_WG # 感染死亡率の直近週間成長率が大きい国

location
Antigua and Barbuda    0.312500
Taiwan                 0.196112
Eritrea                0.144531
Maldives               0.135801
Malaysia               0.111044
Trinidad and Tobago    0.108050
Thailand               0.106407
Nepal                  0.099644
Sri Lanka              0.094322
Papua New Guinea       0.082600
Vietnam                0.062047
Jamaica                0.059980
Namibia                0.051352
Cameroon               0.036832
Sudan                  0.034127
Suriname               0.031148
Singapore              0.029700
Paraguay               0.028661
Bahrain                0.027505
Angola                 0.025531
Uruguay                0.024794
Haiti                  0.017605
Guinea-Bissau          0.014111
Ukraine                0.013180
Cuba                   0.012917
Gabon                  0.012310
Qatar                  0.011358
Russia                 0.010829
Honduras               0.010819
Oman                   0.010515
Name: case_death_rate_weekly_gr